In [3]:
%matplotlib inline
import pandas as pd
import numpy as np
import random as rnd
import seaborn as sns
import matplotlib.pyplot as plt

In [325]:
data = pd.read_csv('train.csv', header=None ,delimiter=";" )
feature_names = ['usuario', 'palabra', 'palabraLeida', 'tiempoCaracter',
       'hayErrPalabra', 'tiempoErrPalabra', 'numPalabra','tiempoPalabra', 'tamPalabra', 'caracter',
       'falloCaracter', 'palabraCorrecta']
data.columns = feature_names

In [302]:
data


,usuario,palabra,palabraLeida,tiempoCaracter,hayErrPalabra,tiempoErrPalabra,numPalabra,tiempoPalabra,tamPalabra,caracter,falloCaracter,palabraCorrecta
0,Cristhian,ROJO,R,1.589,False,0.0,0,0.000,4,R,False,False
1,Cristhian,ROJO,RO,1.733,False,0.0,0,0.000,4,O,False,False
2,Cristhian,ROJO,ROJO,1.865,False,0.0,0,0.000,4,O,False,False
3,Cristhian,ROJO,ROJO,1.971,False,0.0,0,0.000,4,O,False,False
4,Cristhian,ROJO,ROJO,0.318,False,0.0,1,5.476,4,NaN,False,True
5,Cristhian,ROJO,ROJO,0.430,False,0.0,0,0.000,4,O,False,False
6,Cristhian,PANTALON,P,1.021,False,0.0,0,0.000,8,P,False,False
7,Cristhian,PANTALON,PAN,1.213,False,0.0,0,0.000,8,N,False,False
8,Cristhian,PANTALON,PANT,1.318,False,0.0,0,0.000,8,T,False,False
9,Cristhian,PANTALON,PANTA,0.298,False,0.0,0,0.000,8,A,False,False


# Pasar valores verdadero y falso a booleanos
hayErrPalabra, falloCaracter, palabraCorrecta

In [326]:
# Pasamos de boolean a un int, 1 para true y 0 para false
for df in [data]:
    df['hayErrPalabra'] = df['hayErrPalabra'].apply(lambda x : 1 if x == 'true' else 0)
    df['falloCaracter'] = df['falloCaracter'].apply(lambda x : 1 if x == 'true' else 0)
    df['palabraCorrecta'] = df['palabraCorrecta'].apply(lambda x : 1 if x == 'true' else 0)

# Quitarle los espacios en blanco al usuario

In [327]:
data["usuario"] = data["usuario"].str.strip()

# Mapear el usuario en un campo usuarioID

In [328]:
data["usuarioID"] = data['usuario'].map({"Cristhian": 0, "Jesus": 1})

# Sacar tiempo medio de escritura del mismo caracter
Hay que quitar los caracteres nulos

In [168]:
caracter = data[~data['caracter'].isnull()][['usuario', 'caracter','tiempoCaracter']]
caracter.groupby(['usuario','caracter']).mean()

tiempoCaracter
usuario   caracter                
Cristhian                 0.248200
          A               0.401083
          C               1.028000
          D               0.221500
          E               0.517000
          H               0.152000
          I               0.775571
          J               0.633500
          L               0.332750
          N               0.506400
          O               0.663316
          P               0.069500
          R               0.718500
          S               0.421000
          T               0.792500
          X               0.755000
          Z               0.378000
Jesus                     0.427750
          A               0.539333
          C               0.342000
          D               0.279000
          E               0.402000
          H               0.280000
          I               0.540000
          J               0.510000
          L               0.192500
          N               0.615375
          O               0.452286
          P               1.041000
          R               0.398000
          S               0.618500
          T               0.327500
          Z               0.447000

# Sacar tiempo medio de pulsado de enter (caracteres nulos)

In [290]:
Enter = data[data['caracter'].isnull()][['usuario','tiempoCaracter']]
Enter.columns = ['usuario', 'tiempoEnter']
Enter = Enter.groupby(['usuario']).mean()
Enter

,tiempoEnter
usuario,
Cristhian,0.458125
Jesus,0.462200


# Usuario, palabra, tiempo

In [265]:
usPalTiempo = data[data['caracter'].isnull()][['usuario', 'palabra', 'tiempoPalabra','tamPalabra']]
usPalTiempo

,usuario,palabra,tiempoPalabra,tamPalabra
4,Cristhian,ROJO,5.476,4
14,Cristhian,PANTALON,3.278,8
25,Cristhian,ZANAHORIA,5.429,9
42,Cristhian,MINERIA DE DATOS,4.010,16
64,Cristhian,CRISTHIANO RONALDO,0.000,18
68,Jesus,ROJO,2.376,4
79,Jesus,PANTALON,3.806,8
91,Jesus,ZANAHORIA,4.922,9
108,Jesus,MINERIA DE DATOS,3.784,16
139,Jesus,CRISTHIANO RONALDO,0.000,18


# Sacar el tiempoPalabra medio de cada palabra del usuario

In [305]:
usPalTiempo2 = usPalTiempo.groupby(['usuario','palabra']).mean()
usPalTiempo2

tiempoPalabra  tamPalabra
usuario   palabra                                      
Cristhian CRISTHIANO RONALDO       0.000000          18
          MINERIA DE DATOS         4.010000          16
          PANTALON                 3.290500           8
          ROJO                     5.786333           4
          ZANAHORIA                5.429000           9
Jesus     CRISTHIANO RONALDO       0.000000          18
          MINERIA DE DATOS         3.784000          16
          PANTALON                 3.806000           8
          ROJO                     2.376000           4
          ZANAHORIA                4.922000           9

# Sacar tiempo medio por caracter por tamaño de palabra

In [320]:
usPalTiempo3 = usPalTiempo.drop(['palabra'], axis=1)

usPalTiempo3 = usPalTiempo3.groupby(['usuario']).mean()
#usPalTiempo3['tiempoMedioCaracter'] = usPalTiempo3['tiempoPalabra'] / usPalTiempo3['tamPalabra']
usPalTiempo3

,tiempoPalabra,tamPalabra
usuario,,
Cristhian,4.172375,8.875
Jesus,2.977600,11.000


In [294]:
usPalTiempo3['tiempoEnter'] = Enter

In [295]:
usPalTiempo3

,tiempoPalabra,tamPalabra,tiempoMedioCaracter,tiempoEnter
usuario,,,,
Cristhian,4.172375,8.875,0.470127,0.458125
Jesus,2.977600,11.000,0.270691,0.462200


### ¿Sacar media de errores (de caracteres incorrectos) por longitud de palabra?

### ¿Media de fallos al introducir las 7 palabras?

In [317]:
data

,usuario,palabra,palabraLeida,tiempoCaracter,hayErrPalabra,tiempoErrPalabra,numPalabra,tiempoPalabra,tamPalabra,caracter,falloCaracter,palabraCorrecta,usuarioID
0,Cristhian,ROJO,R,1.589,0,0.0,0,0.000,4,R,0,0,0
1,Cristhian,ROJO,RO,1.733,0,0.0,0,0.000,4,O,0,0,0
2,Cristhian,ROJO,ROJO,1.865,0,0.0,0,0.000,4,O,0,0,0
3,Cristhian,ROJO,ROJO,1.971,0,0.0,0,0.000,4,O,0,0,0
4,Cristhian,ROJO,ROJO,0.318,0,0.0,1,5.476,4,NaN,0,0,0
5,Cristhian,ROJO,ROJO,0.430,0,0.0,0,0.000,4,O,0,0,0
6,Cristhian,PANTALON,P,1.021,0,0.0,0,0.000,8,P,0,0,0
7,Cristhian,PANTALON,PAN,1.213,0,0.0,0,0.000,8,N,0,0,0
8,Cristhian,PANTALON,PANT,1.318,0,0.0,0,0.000,8,T,0,0,0
9,Cristhian,PANTALON,PANTA,0.298,0,0.0,0,0.000,8,A,0,0,0


In [329]:
data.describe()

,tiempoCaracter,hayErrPalabra,tiempoErrPalabra,numPalabra,tiempoPalabra,tamPalabra,falloCaracter,palabraCorrecta,usuarioID
count,162.000000,162.0,162.0,162.000000,162.000000,162.000000,162.0,162.0,162.000000
mean,0.504951,0.0,0.0,0.197531,0.297944,8.160494,0.0,0.0,0.462963
std,0.373298,0.0,0.0,0.754609,1.148431,2.434007,0.0,0.0,0.500173
min,-0.882000,0.0,0.0,0.000000,0.000000,4.000000,0.0,0.0,0.000000
25%,0.276250,0.0,0.0,0.000000,0.000000,7.000000,0.0,0.0,0.000000
50%,0.413500,0.0,0.0,0.000000,0.000000,8.000000,0.0,0.0,0.000000
75%,0.614250,0.0,0.0,0.000000,0.000000,10.000000,0.0,0.0,1.000000
max,1.971000,0.0,0.0,4.000000,6.294000,18.000000,0.0,0.0,1.000000


# Sacar el target

In [330]:
target = data['usuarioID']
target

0      0
1      0
2      0
3      0
4      0
5      0
6      0
7      0
8      0
9      0
10     0
11     0
12     0
13     0
14     0
15     0
16     0
17     0
18     0
19     0
20     0
21     0
22     0
23     0
24     0
25     0
26     0
27     0
28     0
29     0
      ..
132    1
133    1
134    1
135    1
136    1
137    1
138    1
139    1
140    0
141    0
142    0
143    0
144    0
145    0
146    0
147    0
148    0
149    0
150    0
151    0
152    0
153    0
154    0
155    0
156    0
157    0
158    0
159    0
160    0
161    0
Name: usuarioID, dtype: int64

# Eliminar campos sobrantes (Usuario, palabra, palabraLeida, numPalabra, tamPalabra, caracter, usuarioID)
Inicialmente palabraCorrecta no, pero no tenemos datos buenos para entrenar, asi que tambien se elimina

In [331]:
data = data.drop(['usuario','palabra','palabraLeida','numPalabra', 'tamPalabra', 'caracter', 'palabraCorrecta', 'usuarioID'], axis=1)

# Cambiar datos malos por las mejoras

In [332]:
data

,tiempoCaracter,hayErrPalabra,tiempoErrPalabra,tiempoPalabra,falloCaracter
0,1.589,0,0.0,0.000,0
1,1.733,0,0.0,0.000,0
2,1.865,0,0.0,0.000,0
3,1.971,0,0.0,0.000,0
4,0.318,0,0.0,5.476,0
5,0.430,0,0.0,0.000,0
6,1.021,0,0.0,0.000,0
7,1.213,0,0.0,0.000,0
8,1.318,0,0.0,0.000,0
9,0.298,0,0.0,0.000,0


# Separar datos de entreno y datos de testeo

# Cross Validation


# Random Forest

In [339]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier

random_forest = RandomForestClassifier(n_estimators=101)
scores = cross_val_score(random_forest, data, target, cv=5)
print(scores)
print(scores.mean())

[ 0.45454545  0.45454545  0.5         0.5625      0.5       ]
0.494318181818


# SVM

In [338]:
from sklearn.model_selection import cross_val_score
from sklearn import svm

svm = svm.SVC(kernel='linear', C=1)
scores = cross_val_score(svm, data, target, cv=5)
print(scores)
print(scores.mean())

[ 0.54545455  0.54545455  0.53125     0.53125     0.53125   ]
0.536931818182


# AdaBoost

In [336]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import cross_val_score

ada = AdaBoostClassifier(n_estimators=100)
scores = cross_val_score(ada, data, target, cv=5)
print(scores)
print(scores.mean())

[ 0.45454545  0.57575758  0.625       0.5625      0.625     ]
0.568560606061
